In [1]:
import numpy as np
import pandas as pd

In [2]:
dataFrame = pd.read_csv("loans_data.csv")
# dataFrame

In [3]:
ColumnLabels = np.array(list(dataFrame))
Train_Matrix = np.matrix(dataFrame.iloc[:, 1:])
Target_Vector = np.array(dataFrame.iloc[:,0])

In [4]:
Grade = Train_Matrix[:, 0]
# Grade

In [5]:
home_ownership = Train_Matrix[:, 4]
# home_ownership

In [6]:
purpose = Train_Matrix[:, 6]
# purpose

## Using 1 of K method to replace categorial feature

In [7]:
Clean_Matrix_un = np.delete(Train_Matrix, [0, 4, 6], 1)
# Clean_Matrix_un

# Normalize Clean_Matrix

In [8]:
from sklearn.preprocessing import minmax_scale,scale
Clean_Matrix_Standard = scale(Clean_Matrix_un)
Clean_Matrix = minmax_scale(Clean_Matrix_Standard)
# Clean_Matrix = Clean_Matrix_Standard
# Clean_Matrix = minmax_scale(Clean_Matrix_un)
# Clean_Matrix

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64.
  warnings.warn(msg, _DataConversionWarning)


## Clean Grade Field Ex: A =>(1,0,0,0,0,0,0)

In [9]:
def clean_grade(grade_matrix):
    new_matrix = np.zeros(shape=(grade_matrix.shape[0], 7))
    for row_number in range(len(grade_matrix)):
        symbol = grade_matrix[row_number, 0]
        arr = np.zeros(7)
        arr[ord(symbol) - ord('A')] = 1
        new_matrix[row_number, :] = arr
    return new_matrix
Clean_Grade_Matrix = clean_grade(Grade)
new_matrix = np.hstack((Clean_Matrix,Clean_Grade_Matrix))

In [10]:
def clean_home_ownership(home_ownership_matrix):
    new_matrix = np.zeros(shape=(home_ownership_matrix.shape[0], 4))
    for row_number in range(len(home_ownership_matrix)):
        symbol = home_ownership_matrix[row_number, 0]
        arr = np.zeros(4)
        if symbol == 'OWN':
            arr[0] = 1
        elif symbol == 'MORTGAGE':
            arr[1] = 1
        elif symbol == 'RENT':
            arr[2] = 1
        elif symbol == 'OTHER':
            arr[3] = 1
        else:
            print(symbol)
        new_matrix[row_number, :] = arr
    return new_matrix
Clean_HomeOwnership_Matrix = clean_home_ownership(home_ownership)
new_matrix2 = np.hstack((new_matrix, Clean_HomeOwnership_Matrix))

In [11]:
def clean_purpose(purpose_matrix):
    new_matrix = np.zeros(shape=(purpose_matrix.shape[0], 12))
    for row_number in range(len(purpose_matrix)):
        symbol = purpose_matrix[row_number, 0]
        arr = np.zeros(12)
        if symbol == 'credit_card':
            arr[0] = 1
        elif symbol == 'medical':
            arr[1] = 1
        elif symbol == 'debt_consolidation':
            arr[2] = 1
        elif symbol == 'car':
            arr[3] = 1
        elif symbol == 'small_business':
            arr[4] = 1
        elif symbol == 'home_improvement':
            arr[5] = 1
        elif symbol == 'vacation':
            arr[6] = 1
        elif symbol == 'major_purchase':
            arr[7] = 1
        elif symbol == 'moving':
            arr[8] = 1
        elif symbol == 'wedding':
            arr[9] = 1
        elif symbol == 'house':
            arr[10] = 1
        elif symbol == 'other':
            arr[11] = 1
        else:
            print(symbol)
        new_matrix[row_number, :] = arr
    return new_matrix
Clean_Purpose_Matrix = clean_purpose(purpose)
Final_Matrix = np.hstack((new_matrix2, Clean_HomeOwnership_Matrix))

### Final Matrix is the result using 1 of k method

In [12]:
Final_Matrix

array([[ 0.5       ,  0.        ,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.27272727, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.5       ,  1.        ,  0.09090909, ...,  0.        ,
         1.        ,  0.        ],
       ..., 
       [ 1.        ,  1.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.5       ,  0.        ,  0.54545455, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  1.        , ...,  1.        ,
         0.        ,  0.        ]])

In [13]:
Target_Vector

array([ 1,  1,  1, ..., -1, -1, -1])

## Start Training SVM

In [14]:
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import accuracy_score

In [52]:
def svm_normal(Final, Target, test_final, test_target,max_iters=1000):
#     clf = SVC(max_iter=max_iters,cache_size=3000,class_weight='balanced',gamma='auto',C=00.1)
    clf = SVC(max_iter=max_iters,cache_size=3000,gamma=0.01,C=1)
    clf.fit(Final, Target)
    Predict_Y = clf.predict(test_final)
    acs = accuracy_score(test_target,Predict_Y)
    return acs

## Using LinearSVC for fast training for linear svm

In [53]:
def svm_linear(Final, Target, test_final, test_target,max_iters=1000):
#     clf2 = SVC(kernel='linear',max_iter=max_iters,class_weight='balanced')
#     clf2 = SVC(kernel='linear',cache_size=3000,max_iter=max_iters)
    clf2 = LinearSVC(max_iter = max_iters 
                     ,C=1
                     , class_weight = 'balanced'
                     ,penalty='l2'
                     ,loss='squared_hinge'
                     ,dual = False
                     ,random_state = 43)
    clf2.fit(Final, Target)
    Predict_Y2 = clf2.predict(test_final)
    acs = accuracy_score(test_target,Predict_Y2)
    return acs

## Using KFold ( K = 12 ) To Evaluate Model Performance

In [20]:
from sklearn.model_selection import KFold
split_num = 12

In [21]:
kf1 = KFold(n_splits=split_num, shuffle=True)
linear_score = 0
linear_iter = 100000
for train, test in kf1.split(Final_Matrix):
    train_feature, test_feature = Final_Matrix[train], Final_Matrix[test]
    train_target, test_target = Target_Vector[train], Target_Vector[test]
    linear_score += svm_linear(train_feature, train_target, test_feature, test_target,linear_iter)

In [ ]:
print('linear kernel',linear_score/split_num)

linear kernel 0.661377137499


In [ ]:
normal_score = 0
normal_iter = 30000
kf2 = KFold(n_splits=split_num, shuffle=True)
for train, test in kf2.split(Final_Matrix):
    train_feature, test_feature = Final_Matrix[train], Final_Matrix[test]
    train_target, test_target = Target_Vector[train], Target_Vector[test]
    normal_score += svm_normal(train_feature, train_target, test_feature, test_target,normal_iter)

In [27]:
print('rbf kernel',normal_score/split_num)

rbf kernel 0.611229673646
